In [1]:
from openai import OpenAI
import os
from dotenv import dotenv_values
from ReadLoad import read_jsonl, write_jsonl, read_json,write_json
from tqdm import tqdm
from evaluation import get_embedding, cosine_similarity
from prompt_template import get_sys_prompt, get_qa_prompt
#from shusheng import  get_respond
from eval_data import key_word_score
from qwen import qwen_response
import pandas as pd
import time

config = dotenv_values('.env')
client = OpenAI(
    api_key=config['qwen_key'],
    base_url=config['qwen_url']
)


def get_response(prompt):
    time.sleep(0.1)
    try:
        completion = client.chat.completions.create(
            model="qwen2-7b-instruct",
            messages=[
                {'role': 'system', 'content': get_sys_prompt()},
                {'role': 'user', 'content': prompt}
            ]

            )
        return completion.choices[0].message.content
    except Exception as e:
    # Handle any other unexpected exceptions
        print(f"An unexpected error occurred: {e}")
        return f"An unexpected error occurred: {e}"


2024-07-21 12:32:06.604592: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 12:32:06.616424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 12:32:06.630553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 12:32:06.634769: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 12:32:06.648729: I tensorflow/core/platform/cpu_feature_guar

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def completion(data):
    for d in tqdm(data):
        query = d['问题']
        clause = d['条款']
        name = d['产品名']
        prompt = get_qa_prompt(name,clause,query)
        d['prompt'] = prompt
        #d['answer'] = get_response(prompt)
        #d['answer'] = get_respond(prompt)
        d['answer'] = qwen_response(prompt)
    return data

In [3]:
def evaluation(data, name="测试数据"):
    similaritys = []
    for d in tqdm(data):
        vec1 = dev_data_embd[d['ID']]
        vec2 = get_embedding(d['answer'])
        similarity = cosine_similarity(vec1, vec2)
        d['similarity'] = similarity
        d['key_word_similarity'] = key_word_score(d['答案'], d['answer'])
        similaritys.append(similarity)
    length = len(data)
    score = sum(similaritys)/length
    print(f'测评数据集：{length} 模型推理结果与答案的相识度得分：{score}')
    df = pd.DataFrame(data)
    df.to_excel(f"{name}_score_{score}.xlsx", index=False)
    return data

In [4]:
def to_summit_json(data, commit="大道至简_result"):
    result = []
    for d in data:
        rd = {
            "ID": d['ID'],
            "question": d['问题'],
            "answer": d['answer']
        }
        result.append(rd)
    #write_json(result, commit)
    write_jsonl(result, commit)

### loading data

In [5]:
#test_data = read_json("dataset/test.json")
dev_data = read_jsonl("dataset/resultdev_with_embedding.jsonl")
dev_data_embd = {}
for d in dev_data:
    dev_data_embd[d['ID']] = d['ans_embedding']
    d.pop('ans_embedding')

In [6]:
# 将数据也移到GPU上
comp_data = completion(dev_data[200:300])
eval_data = evaluation(comp_data)

# test_data_result = completion(test_data)
# to_summit_json(test_data_result)


  0%|          | 0/100 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.576 seconds.
Prefix dict has been built successfully.
100%|██████████| 100/100 [00:23<00:00,  4.28it/s]


测评数据集：100 模型推理结果与答案的相识度得分：0.9169045935792026


In [8]:
 get_qa_prompt("","","")

'产品名称：""""""\n保险条款：""""""\n用户问题：""""""\n回答规则："""回答用户问题时，严格依据产品名称和保险条款中的内容提供精确且清晰的回答。\n如果在保险条款中找不到答案，告诉用户在段落内容中并未提供具体的答案，并描述所提及的内容。"""\n回答:'

In [7]:
from qwen import qwen_response
qwen_response("你好")

'你好！有什么关于保险条款的问题我可以帮助解答吗？'

### 参考资料

https://dashscope.console.aliyun.com/billing